In [1]:
from torch_uncertainty import cli_main, init_args
from torch_uncertainty.baselines.classification import ResNet
from torch_uncertainty.optimization_procedures import optim_cifar10_resnet18
from torch_uncertainty.datamodules import CIFAR10DataModule
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.nn import CrossEntropyLoss
import torch

import os
from pathlib import Path
from cli_test_helpers import ArgvContext

In [2]:
torch.cuda.is_available()

True

### 1 - Models

In [3]:
baseline = ResNet(num_classes=10,
                loss=CrossEntropyLoss,
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3, 
                arch=18).cuda()

mixup = ResNet(num_classes=10,
                loss=CrossEntropyLoss,
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3, 
                arch=18, 
                mixup=True,
                mixup_alpha=0.2).cuda()

regmixup = ResNet(num_classes=10,
                loss=CrossEntropyLoss,
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3,
                arch=18,
                reg_mixup=True,
                mixup_alpha=15).cuda()

### 2 - Data

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 3. Training

In [9]:
root = Path(os.path.abspath(""))

# We mock the arguments for the trainer
with ArgvContext(
    "file.py",
    "--max_epochs",
    "20",
    "--enable_progress_bar",
    "False",
    "--num_estimators",
    "8"
):
    args = init_args(network=ResNet, datamodule=CIFAR10DataModule)

net_name = "logs/reset18-cifar10"

# datamodule
args.root = str(root / "data")
dm = CIFAR10DataModule(**vars(args))


In [10]:
#Trainer(accelerator="gpu", devices=1)
results = cli_main(baseline, dm, root, net_name, args=args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | model            | _ResNet          | 11.2 M
1 | format_batch_fn  | Identity         | 0     
2 | val_cls_metrics  | MetricCollection | 0     
3 | test_cls_metrics | MetricCollection | 0     
4 | test_entropy_id  | Entropy          | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_10\checkpoints\epoch=16-step=6647.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_10\checkpoints\epoch=16-step=6647.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      cls_test/acc          0.7439000010490417
     cls_test/brier         0.3573360741138458
      cls_test/ece         0.033778075128793716
    cls_test/entropy        0.6558901071548462
      cls_test/nll          0.7499282956123352
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [11]:
results = cli_main(mixup, dm, root, net_name, args=args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | model            | _ResNet          | 11.2 M
1 | format_batch_fn  | Identity         | 0     
2 | val_cls_metrics  | MetricCollection | 0     
3 | test_cls_metrics | MetricCollection | 0     
4 | test_entropy_id  | Entropy          | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_11\checkpoints\epoch=19-step=7820.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_11\checkpoints\epoch=19-step=7820.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      cls_test/acc          0.7475000023841858
     cls_test/brier         0.35359084606170654
      cls_test/ece          0.05278158560395241
    cls_test/entropy        0.5841179490089417
      cls_test/nll          0.7453628778457642
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [13]:
results = cli_main(regmixup, dm, root, net_name, args=args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params
------------------------------------------------------
0 | model            | _ResNet          | 11.2 M
1 | format_batch_fn  | Identity         | 0     
2 | val_cls_metrics  | MetricCollection | 0     
3 | test_cls_metrics | MetricCollection | 0     
4 | test_entropy_id  | Entropy          | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_13\checkpoints\epoch=12-step=5083.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at d:\COURS\A5\Responsible AI\Code\logs/reset18-cifar10\version_13\checkpoints\epoch=12-step=5083.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      cls_test/acc          0.7820000052452087
     cls_test/brier         0.3092312216758728
      cls_test/ece          0.02364463172852993
    cls_test/entropy         0.590100884437561
      cls_test/nll          0.6474164128303528
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
